In [8]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from neo4j.exceptions import ConfigurationError
from langchain_huggingface import HuggingFaceEmbeddings
from src.grag.retrievers.hybrid_cypher.hybrid_cypher import create_hybrid_cypher_retriever_tool

load_dotenv(".env")

True

In [ ]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]
DATABASE = os.environ["DATABASE_SMALL"]

neo4j_config = {
    "DATABASE_NAME": DATABASE,
    "ARTICLE_VECTOR_INDEX_NAME": os.environ["ARTICLE_VECTOR_INDEX_NAME"],
    "ARTICLE_FULLTEXT_INDEX_NAME": os.environ["ARTICLE_FULLTEXT_INDEX_NAME"],
    "DEFINITION_VECTOR_INDEX_NAME": os.environ["DEFINITION_VECTOR_INDEX_NAME"],
    "DEFINITION_FULLTEXT_INDEX_NAME": os.environ["DEFINITION_FULLTEXT_INDEX_NAME"],
}

# Ngga jadi pakai ini karena Text2Cypher mintannya harus pakai Neo4jGraph
# neo4j_driver = GraphDatabase.driver(uri=URI, auth=(USERNAME, PASSWORD))

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)
neo4j_driver = neo4j_graph._driver  # Ambil driver nya kaya gini
embedder_model = HuggingFaceEmbeddings(model_name=os.environ["EMBEDDING_MODEL"])

try:
    neo4j_driver.verify_connectivity()
except ConfigurationError:
    raise ValueError(
        "Could not connect to Neo4j database. "
        "Please ensure that the driver config is correct"
    )

In [10]:
hybrid_cypher_retriever = create_hybrid_cypher_retriever_tool(
    embedder_model=embedder_model,
    neo4j_driver=neo4j_driver,
    neo4j_config=neo4j_config,
    top_k_initial_article=5,
    total_article_limit=10,
    total_definition_limit=5
)

In [11]:
query = "Apa itu penyelenggara sistem elektronik?"

In [12]:
result = hybrid_cypher_retriever.invoke({"query": query})
print(result)

## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**
--------------------------------------------------------------------------------

Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 30:
(1) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik milik Orang lain dengan cara apa pun.
(2) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik dengan cara apa pun dengan tujuan untuk memperoleh Informasi Elektronik dan/atau Dokumen Elektronik.
(3) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer y^dan/atau Sistem Elektronik dengan cara apa pun dengan melanggar, menerobos, melampaui, atau menjebol sistem pengamanan.

Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik, BAB II - PENYELENGGARAAN 

In [13]:
# Agar hasilnya dalam ToolMessage, harus invoke menggunakan schema ToolCall
# https://python.langchain.com/docs/how_to/tool_artifacts/#invoking-the-tool-with-toolcall

result = hybrid_cypher_retriever.invoke(
    {
        "name": "hybrid_cypher_retriever",
        "args": {"query": query},
        "id": "123",  # required
        "type": "tool_call",  # required
    }
)

result.model_dump()

{'content': '## **Daftar Pasal Peraturan Perundang-Undangan yang (Mungkin) Relevan untuk Menjawab Kueri:**\n--------------------------------------------------------------------------------\n\nUndang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 30:\n(1) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik milik Orang lain dengan cara apa pun.\n(2) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer dan/atau Sistem Elektronik dengan cara apa pun dengan tujuan untuk memperoleh Informasi Elektronik dan/atau Dokumen Elektronik.\n(3) Setiap Orang dengan sengaja dan tanpa hak atau melawan hukum mengakses Komputer y^dan/atau Sistem Elektronik dengan cara apa pun dengan melanggar, menerobos, melampaui, atau menjebol sistem pengamanan.\n\nPeraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik, BAB 

In [14]:
len(result.content)

7470